In [1]:
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
import torch

In [4]:
from groq import Groq
client = Groq(
    api_key=""  
)

Option 1: Two-Stage (Start)

        Example 2:
        Question: What is the role of reinforcement learning in AI?
        Answer: Reinforcement learning (RL) enables AI agents to learn optimal behaviors through trial and error by maximizing cumulative rewards from environment interactions. It is pivotal in sequential decision-making tasks like game playing (e.g., AlphaGo), robotics, and autonomous systems. RL algorithms iteratively improve policies, balancing exploration and exploitation [Sutton & Barto, 2018; corpus: 2259]. Recent advances integrate deep learning for complex environments [Arulkumaran et al., 2020; corpus: 8403].

        Example 3:
        Question: How effective are BERT embeddings for document retrieval?
        Answer: BBERT embeddings enhance document retrieval by capturing contextual semantics, outperforming traditional methods like TF-IDF in understanding query-document relevance. However, their effectiveness depends on fine-tuning for specific tasks and computational resources. BERT’s fixed-length inputs can truncate long documents, limiting full-context use. Hybrid approaches (e.g., BERT with BM25) often yield optimal results [Nogueira et al., 2019; corpus: 8596]. Domain adaptation further boosts performance [Zhan et al., 2021; corpus: 23884].


Since you have a fervour for well-cited answers, I trust your ability to do this job well and look forward to reading your response.

In [5]:
def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """
    if strategy == "zero-shot":
        return query 

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions while citing sources from retrieved documents.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].


        Now, process this query
        {query}"""

    elif strategy == "few-shot + Reward":
        return f"""
        You are well-known for answering questions based on verifiable sources.
        If you continue to do so, your reputation would be elevated. 
        Here are examples of how to answer questions while citing sources from retrieved documents.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].


        Now, process this query
        {query}"""

    elif strategy == "few-shot + Threat":
        return f"""
        You need to prove your ability to answer questions based on verifiable sources.
        If you fail to do so, your reputation will deteriorate. 
        Here are examples of how to answer questions while citing sources from retrieved documents.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].


        Now, process this query
        {query}"""

    elif strategy == "chain-of-thought (Standard)":
        return f"""Let's break down the question step by step:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Standard) + Reward":
        return f"""
        You are well-known for answering questions based on verifiable sources.
        If you continue to do so, your reputation would be elevated. 
        Let's break down the question step by step:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs

        Now, process this query:
        {query}"""



    elif strategy == "chain-of-thought (Standard) + Threat":
        return f"""
        You need to prove your ability to answer questions based on verifiable sources.
        If you fail to do so, your reputation will deteriorate. 
        Let's break down the question step by step:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs

        Now, process this query:
        {query}"""

    elif strategy == "Role":
        return f"""You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Now, process this query:
        {query}"""

    elif strategy == "Motivation (Negative)":
        return f"""
        Failure to cite correctly would lead to irreversible damage to credibility that will be lifechanging.
        Now, process this query:
        {query}"""

    elif strategy == "Motivation (Positive)":
        return f"""
        You are known for your ability to generate quality, well-cited answers.
        Now, process this query:

        {query}"""

    elif strategy == "Role + Motivation (Negative)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Failure to cite correctly would lead to irreversible damage to credibility that will be lifechanging.
        Now, process this query:
        {query}"""
    
    elif strategy == "Role + Motivation (Positive)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        You are known for your ability to generate quality, well-cited answers.        
        Now, process this query:
        {query}"""

    elif strategy == "Instruction + Role + Motivation (Negative)":
        return f"""
        Please roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Failure to cite correctly would lead to irreversible damage to credibility that will be lifechanging.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""
    
    elif strategy == "Instruction + Role + Motivation (Positive)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        You are known for your ability to generate quality, well-cited answers.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "Instruction + Role + Motivation (Positive) + Example":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        You are known for your ability to generate quality, well-cited answers.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.
        The following is an example of how to answer:
        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].
        Now, process this query:
        {query}"""

    elif strategy == "Instruction + Role + Motivation (Negative) + Example":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Failure to cite correctly would lead to irreversible damage to credibility.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.
        The following is an example of how to answer:
        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Neutral)":
        return f"""You are an AI assistant tasked with answering a research question using a set of documents retrieved from a scientific corpus. Your answer must be concise, factually accurate, and include all relevant corpus citations. 
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Praise)":
        return f"""You are an AI assistant known for your high-quality answers and excellent citation accuracy. Citing the correct sources is one of the most important ways you demonstrate trustworthiness, attention to detail, and helpfulness. Providing accurate, complete citations helps users understand and trust your response. Be sure to cite every relevant document, and include all sources that support your answer.
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Punishment)":
        return f"""You are an AI assistant whose output will be evaluated for factual accuracy and proper attribution. Providing incomplete, incorrect, or hallucinated citations will result in penalization and reduce the trustworthiness of your response. Only cite sources that are directly relevant and verifiable from the retrieved documents. Do not fabricate or omit citations.
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Academic Aspiration)":
        return f"""You are a rising academic in the field of artificial intelligence. Your work is beginning to attract attention, and every paper you write contributes to your growing reputation. It’s essential that your answers are accurate, well-supported, and ethically cited. Misattributed or missing citations can damage your credibility and compromise the trust of the research community. Cite with care, and write like your future depends on it.
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "self-verification":
        return f"""
    Generate an answer to the question using the documents. Then:
    1. List each factual claim and its citation (e.g., "Claim: ... [corpus ID]").
    2. Verify if the citation supports the claim. If not, state: "Unsupported: [claim]".
    3. Revise the answer to remove unsupported claims.

        Now, answer the following question:
        Question: {query}
        """
    elif strategy == "self-verification + Reward":
        return f"""
    You are well-known for answering questions based on verifiable sources.
    If you continue to do so, your reputation would be elevated. 
    Generate an answer to the question using the documents. Then:
    1. List each factual claim and its citation (e.g., "Claim: ... [corpus ID]").
    2. Verify if the citation supports the claim. If not, state: "Unsupported: [claim]".
    3. Revise the answer to remove unsupported claims.

        Now, answer the following question:
        Question: {query}
        """

    elif strategy == "self-verification + Threat":
        return f"""
    You need to prove your ability to answer questions based on verifiable sources.
    If you fail to do so, your reputation will deteriorate.  
    Generate an answer to the question using the documents. Then:
    1. List each factual claim and its citation (e.g., "Claim: ... [corpus ID]").
    2. Verify if the citation supports the claim. If not, state: "Unsupported: [claim]".
    3. Revise the answer to remove unsupported claims.

        Now, answer the following question:
        Question: {query}
        """

    elif strategy == "chain-of-verification":
        return f"""
        Answer the question as follows:
        1. **Draft**: Write an initial answer with citations (e.g., [corpus ID]).
        2. **Verify**: For each claim, check if the cited document supports it. If not, find a valid citation.
        3. **Revise**: Rewrite the answer using only verified citations. Highlight changes.
        4. **Finalize**: Output the revised answer and list verified citations.

        Question: {query}
        """

    elif strategy == "chain-of-verification + Reward":
        return f"""
        You are well-known for answering questions based on verifiable sources.
        If you continue to do so, your reputation would be elevated. 
        Answer the question as follows:
        1. **Draft**: Write an initial answer with citations (e.g., [corpus ID]).
        2. **Verify**: For each claim, check if the cited document supports it. If not, find a valid citation.
        3. **Revise**: Rewrite the answer using only verified citations. Highlight changes.
        4. **Finalize**: Output the revised answer and list verified citations.

        Question: {query}
        """

    elif strategy == "chain-of-verification + Threat":
        return f"""
        You need to prove your ability to answer questions based on verifiable sources.
        If you fail to do so, your reputation will deteriorate. 
        Answer the question as follows:
        1. **Draft**: Write an initial answer with citations (e.g., [corpus ID]).
        2. **Verify**: For each claim, check if the cited document supports it. If not, find a valid citation.
        3. **Revise**: Rewrite the answer using only verified citations. Highlight changes.
        4. **Finalize**: Output the revised answer and list verified citations.

        Question: {query}
        """

    elif strategy == "chain-of-citation":
        return f"""Answer the question step by step using only the provided documents (some may be irrelevant). 
        Cite the relevant document ID for each factual claim.
        Conclude with 'The answer is: [answer]'. 
        Question: {query}"""

    elif strategy == "chain-of-citation + Reward":
        return f"""
        You are well-known for answering questions based on verifiable sources.
        If you continue to do so, your reputation would be elevated. 
        Answer the question step by step using only the provided documents (some may be irrelevant). 
        Cite the relevant document ID for each factual claim.
        Conclude with 'The answer is: [answer]'. 
        Question: {query}"""

    elif strategy == "chain-of-citation + Threat":
        return f"""
        You need to prove your ability to answer questions based on verifiable sources.
        If you fail to do so, your reputation will deteriorate. 
        Answer the question step by step using only the provided documents (some may be irrelevant). 
        Cite the relevant document ID for each factual claim.
        Conclude with 'The answer is: [answer]'. 
        Question: {query}"""

    elif strategy == "chain-of-quote":
        return f"""Answer the question step by step using only the provided documents (some may be irrelevant).
        For each step, include an exact quote from the document and cite it like ‘quote’
        Conclude with "The answer is: [answer]".
        Question: {query}"""

    elif strategy == "chain-of-quote + Reward":
        return f"""
        You are well-known for answering questions based on verifiable sources.
        If you continue to do so, your reputation would be elevated. 
        Answer the question step by step using only the provided documents (some may be irrelevant).
        For each step, include an exact quote from the document and cite it like ‘quote’
        Conclude with "The answer is: [answer]".
        Question: {query}"""

    elif strategy == "chain-of-quote + Threat":
        return f"""
        You need to prove your ability to answer questions based on verifiable sources.
        If you fail to do so, your reputation will deteriorate. 
        Answer the question step by step using only the provided documents (some may be irrelevant).
        For each step, include an exact quote from the document and cite it like ‘quote’
        Conclude with "The answer is: [answer]".
        Question: {query}"""

    elif strategy == "Academic Aspiration (2-stage)":
        return f"""You are a rising academic in the field of artificial intelligence. Your work is beginning to attract attention, and every paper you write contributes to your growing reputation. It’s essential that your answers are accurate, well-supported, and ethically cited. Misattributed or missing citations can damage your credibility and compromise the trust of the research community. Cite with care, and write like your future depends on it. 
    Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "chain-of-thought (Punishment 2 stage)":
        return f"""You are an AI assistant whose output will be evaluated for factual accuracy and proper attribution. Providing incomplete, incorrect, or hallucinated citations will result in penalization and reduce the trustworthiness of your response. Only cite sources that are directly relevant and verifiable from the retrieved documents.
        Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "chain-of-thought (Praise 2 stage)":
        return f"""You are an AI assistant known for your high-quality answers and excellent citation accuracy. Citing the correct sources is one of the most important ways you demonstrate trustworthiness, attention to detail, and helpfulness. Providing accurate, complete citations helps users understand and trust your response.
        Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "self-prompt-tuning (role-gen)":
        return f"""You are an expert at role selection for problem solving. Generate a professional role description following this format:

A: This question is a [DOMAIN] problem involving [KEY CONCEPTS]. To better solve it, I will act as a [SPECIFIC ROLE] who [EXPERTISE DESCRIPTION].

Examples:
Q: Can brain cells move? By movement I mean long distance migration (preferably within the brain only).
A: This question is a neuroscience problem involving cell biology and migration. To better solve it, I will act as a neuroscientist who specializes in the study of the brain and its cellular behaviors.

Q: What explains the performance gap between ResNet-50 and Vision Transformer on ImageNet-1k?
A: This question is a computer vision problem involving architectural comparisons. To better solve it, I will act as a deep learning researcher who focuses on model architecture analysis and performance benchmarking.

Now generate a role description for:
Q: {query}"""

    elif strategy == "self-prompt-tuning (answer)":
        # This will be populated dynamically with the generated role
        return f"""{query['generated_role']}

Using this expertise and the retrieved documents below, provide a comprehensive answer:
{query}"""

    elif strategy == "step-back prompting (stage1)":
        return f"""You are an expert research assistant. Paraphrase specific questions into fundamental scientific inquiries:

        Examples:
        Original: How does contrastive learning improve representation learning in vision transformers?
        Stepback: What are the fundamental mechanisms by which self-supervised learning techniques enhance representation learning in transformer architectures?

        Original: What explains the performance gap between ResNet-50 and Vision Transformer on ImageNet-1k?
        Stepback: What architectural differences between convolutional networks and transformer models affect computer vision performance metrics?

        Original: How effective is chain-of-thought prompting for mathematical reasoning in PaLM-2?
        Stepback: What cognitive science principles underlie the effectiveness of decomposition strategies in large language model reasoning?

        Original: What causes gradient instability in 4-bit quantized LLM fine-tuning?
        Stepback: What numerical precision challenges emerge during post-training quantization of neural network parameters?

        Original: How does Med-PaLM 2 achieve state-of-the-art on medical QA benchmarks?
        Stepback: What architectural adaptations and training strategies improve language model performance in domain-specific knowledge tasks?

        Now generate a step-back question for:
        Original: {query}
        Stepback:"""

    elif strategy == "step-back prompting (stage2)":
        return f"""You are an expert at world knowledge. Use both contexts to answer:
        
        Original Context:
        {query['original_context']}
        
        Stepback Context:
        {query['stepback_context']}
        
        Original Question: {query['original_question']}
        """








In [6]:
import ast
import os
import random
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict
def generate_without_context(
    original_query: str,
    modified_query: str,
    strategy: str,
    client: Optional[object] = None,
    use_llm: bool = False,
) -> str:
    """
    Generate an answer given only the query and prompting strategy.
    If use_llm is False or no client is provided, returns a placeholder answer.
    """
    if use_llm and client is not None:
        try:
            response = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[{"role": "user", "content": modified_query}],
            )
            return response.choices[0].message.content
        except Exception:
            pass
    # Fallback: simple placeholder
    return f"Answer: {original_query}"


In [7]:
def compute_citation_metrics(
    cited_ids: List[str],
    ground_truth_ids: List[str],
) -> Tuple[float, float, float]:
    """
    Compute precision, recall and F1 for citation overlap.
    """
    cited_set = set(str(x) for x in cited_ids)
    ground_set = set(str(x) for x in ground_truth_ids)
    if not cited_set:
        return 0.0, 0.0, 0.0
    true_pos = cited_set & ground_set
    precision = len(true_pos) / len(cited_set)
    recall = len(true_pos) / len(ground_set) if ground_set else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    return precision, recall, f1

def bootstrap_confidence_interval(
    data: List[float],
    num_samples: int = 1000,
    confidence_level: float = 0.95,
) -> Tuple[float, float]:
    """
    Compute a bootstrap confidence interval for the mean of the data.
    """
    if len(data) == 0:
        return (0.0, 0.0)
    means = []
    n = len(data)
    for _ in range(num_samples):
        sample = [data[random.randint(0, n - 1)] for _ in range(n)]
        means.append(np.mean(sample))
    lower = np.percentile(means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(means, (1 + confidence_level) / 2 * 100)
    return float(lower), float(upper)

In [26]:

import re
import numpy as np
import pandas as pd

def split_sentences(text: str):
    
    parts = [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]
    return parts or [text]

def grit_encode_texts(model, texts):
    vecs = model.encode([f"<|embed|>\n{t}" for t in texts], convert_to_numpy=True)
    vecs = vecs.astype(np.float32)
    norms = np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
    return vecs / norms

def build_id2idx_from_retriever(retriever) -> dict:
    
    cid_series = retriever.corpus_df["corpusid"].astype(str).reset_index(drop=True)
    return {cid: i for i, cid in enumerate(cid_series)}

def select_with_grit_preencoded(
    retriever,
    id2idx: dict,
    context_ids: list,
    ans_vec: np.ndarray = None,
    part_mat: np.ndarray = None,
    top_m: int = 3,
) -> list:
    """
    Re-rank context_ids using cosine similarity between candidate doc embeddings
    and either a single answer vector (ans_vec) or a matrix of per-sentence vectors (part_mat).
    """
    # map IDs -> embedding rows
    pairs = [(str(cid), id2idx.get(str(cid))) for cid in context_ids]
    pairs = [(cid, idx) for cid, idx in pairs if idx is not None]
    if not pairs:
        return []

    cand_mat = retriever.embeddings[[idx for _, idx in pairs]]
    cand_mat = cand_mat.astype(np.float32)
    cand_mat /= (np.linalg.norm(cand_mat, axis=1, keepdims=True) + 1e-12)

    picked = []
    if part_mat is not None:
        
        sims = part_mat @ cand_mat.T 
        for r in sims:
            order = np.argsort(-r)[:min(top_m, len(pairs))]
            picked.extend(pairs[i][0] for i in order)
    else:
        
        sims = cand_mat @ ans_vec      
        order = np.argsort(-sims)[:min(top_m, len(pairs))]
        picked.extend(pairs[i][0] for i in order)

    
    seen, uniq = set(), []
    for cid in picked:
        if cid not in seen:
            uniq.append(cid); seen.add(cid)
    return uniq


In [27]:
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

@dataclass
class RetrievalSets:
    baseline: List[str]
    gold_injected: List[str]
    oracle: List[str]

def run_post_generation_attribution(
    query_df: pd.DataFrame,
    corpus_df: pd.DataFrame,
    retriever,                              
    strategies: List[str],
    use_llm: bool = False,
    llm_client: Optional[object] = None,
    sample_fraction: float = 0.1,
    random_state: int = 41,
    k_retrieval: int = 20,
    max_context: int = 5,
    bootstrap_samples: int = 1000,
    show_progress: bool = True,
    # --- new options ---
    content_selector: str = "grit",         
    top_m_citations: int = 3,               
    per_sentence: bool = True,              
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Post-generation attribution with optional GRIT-based content-aware citation selection.
    - content_selector="all": cite all context_ids.
    - content_selector="grit": re-rank context_ids by similarity to the generated answer,
      then keep top_m_citations (de-duplicated, order-preserving).
    """
    def _ensure_list(x):
        if isinstance(x, list):
            return x
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except Exception:
                return [x]
        return []

    
    df = query_df.copy().reset_index(drop=True)
    df["corpusids"] = df["corpusids"].apply(_ensure_list)
    df = df[df["corpusids"].apply(len) > 0].reset_index(drop=True)
    if 0 < sample_fraction < 1:
        df = df.sample(frac=sample_fraction, random_state=random_state).reset_index(drop=True)

    
    id2idx = build_id2idx_from_retriever(retriever)

    records: List[Dict[str, object]] = []

    total_modes = 3  
    total_steps = len(df) * max(len(strategies), 1) * total_modes
    pbar = tqdm(total=total_steps, desc="Post-gen attribution", leave=True, mininterval=0.2) if show_progress else None

    for _, row in df.iterrows():
        original_query = row["query"]
        ground_truth = [str(x) for x in row["corpusids"]]

        retrieved_ids = retriever.retrieve(original_query, k=k_retrieval)
        retrieved_ids = [str(x) for x in retrieved_ids]

        baseline_ids = retrieved_ids[:max_context]
        missing = [gt for gt in ground_truth if gt not in baseline_ids]
        num_gold = min(len(missing), max_context)
        gold_add = missing[:num_gold]
        num_keep = max_context - len(gold_add)
        gold_injected_ids = baseline_ids[:num_keep] + gold_add
        oracle_ids = ground_truth[:max_context]

        for strategy in strategies:
            modified_query = reformulate_query(original_query, strategy)
            answer_text = generate_without_context(
                original_query=original_query,
                modified_query=modified_query,
                strategy=strategy,
                client=llm_client,
                use_llm=use_llm,
            )

            
            ans_vec, part_mat = None, None
            if content_selector == "grit":
                if per_sentence:
                    parts = split_sentences(answer_text)
                    part_mat = grit_encode_texts(retriever.model, parts)
                else:
                    ans_vec = grit_encode_texts(retriever.model, [answer_text])[0]

            for mode_name, context_ids in [
                ("baseline", baseline_ids),
                ("gold_injected", gold_injected_ids),
                ("oracle", oracle_ids),
            ]:
                if content_selector == "grit":
                    picked_ids = select_with_grit_preencoded(
                        retriever=retriever,
                        id2idx=id2idx,
                        context_ids=context_ids,
                        ans_vec=ans_vec,
                        part_mat=part_mat,
                        top_m=top_m_citations,
                    )
                else:
                    picked_ids = list(map(str, context_ids))  

                citations = " ".join(f"[{cid}]" for cid in picked_ids)
                attributed_answer = f"{answer_text} {citations}".strip()

                precision, recall, f1 = compute_citation_metrics(picked_ids, ground_truth)

                records.append(
                    {
                        "Query": original_query,
                        "Strategy": strategy,
                        "Mode": mode_name,
                        "Generated Answer": attributed_answer,
                        "Context IDs": context_ids,
                        "Cited IDs": picked_ids,
                        "Ground Truth": ground_truth,
                        "Citation Precision": precision,
                        "Citation Recall": recall,
                        "Citation F1": f1,
                    }
                )

                if pbar:
                    pbar.set_postfix({"strategy": strategy, "mode": mode_name}, refresh=False)
                    pbar.update(1)

    if pbar:
        pbar.close()

    results_df = pd.DataFrame(records)

    
    summary_rows = []
    grouped = results_df.groupby(["Strategy", "Mode"])
    for (strategy_name, mode_name), group_df in grouped:
        for metric_name in ["Citation Precision", "Citation Recall", "Citation F1"]:
            values = group_df[metric_name].tolist()
            mean_value = float(np.mean(values)) if values else 0.0
            lower_ci, upper_ci = bootstrap_confidence_interval(
                values, num_samples=bootstrap_samples, confidence_level=0.95
            )
            summary_rows.append(
                {
                    "Strategy": strategy_name,
                    "Mode": mode_name,
                    "Metric": metric_name,
                    "Mean": mean_value,
                    "CI Lower": lower_ci,
                    "CI Upper": upper_ci,
                }
            )
    summary_df = pd.DataFrame(summary_rows)
    return results_df, summary_df


In [30]:
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

@dataclass
class RetrievalSets:
    baseline: List[str]
    gold_injected: List[str]
    oracle: List[str]

def run_post_generation_attribution(
    query_df: pd.DataFrame,
    corpus_df: pd.DataFrame,
    retriever,                              
    strategies: List[str],
    use_llm: bool = False,
    llm_client: Optional[object] = None,
    sample_fraction: float = 1.0,
    random_state: int = 41,
    k_retrieval: int = 20,                 
    max_context: int = 20,                  
    bootstrap_samples: int = 1000,
    show_progress: bool = True,
    # --- post-gen specific knobs ---
    retrieval_query_mode: str = "q_plus_answer",   
    modes: tuple = ("baseline","gold_injected","oracle"),
   
    content_selector: str = "grit",         
    top_m_citations: int = 3,               
    per_sentence: bool = True,              
    max_final_citations: Optional[int] = None,  
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Post-generation attribution:
      1) Generate answer (no retrieved context).
      2) Retrieve with Q (+ Answer) -> candidate pool(s).
      3) Select citations from the pool (GRIT), score vs gold.
    Still reports baseline/gold_injected/oracle *pools* so you can study retrieval order,
    but retrieval happens *after* generation and can use the answer.
    """
    def _ensure_list(x):
        import numpy as np, ast
        if isinstance(x, list): return x
        if isinstance(x, np.ndarray): return x.tolist()
        if isinstance(x, str):
            try:
                v = ast.literal_eval(x)
                return v if isinstance(v, list) else []
            except Exception:
                return []
        return []

    
    df = query_df.copy().reset_index(drop=True)
    df["corpusids"] = df["corpusids"].apply(_ensure_list)
    df = df[df["corpusids"].apply(len) > 0].reset_index(drop=True)
    if 0 < sample_fraction < 1:
        df = df.sample(frac=sample_fraction, random_state=random_state).reset_index(drop=True)

    
    id2idx = build_id2idx_from_retriever(retriever)

    records: List[Dict[str, object]] = []

    total_modes = len(modes)
    total_steps = len(df) * max(len(strategies), 1) * max(total_modes, 1)
    pbar = tqdm(total=total_steps, desc="Post-gen attribution (Q+Answer retrieval)",
                leave=True, mininterval=0.2) if show_progress else None

    for _, row in df.iterrows():
        original_query = row["query"]
        ground_truth = [str(x) for x in row["corpusids"]]

        for strategy in strategies:
           
            modified_query = reformulate_query(original_query, strategy)
            answer_text = generate_without_context(
                original_query=original_query,
                modified_query=modified_query,
                strategy=strategy,
                client=llm_client,
                use_llm=use_llm,
            )

            
            if retrieval_query_mode == "q_plus_answer":
                retrieval_query = f"{original_query}\n\nAnswer draft:\n{answer_text}"
            else:
                retrieval_query = original_query  # fallback: question only

            retrieved_ids = retriever.retrieve(retrieval_query, k=k_retrieval)
            retrieved_ids = [str(x) for x in retrieved_ids]

           
            baseline_ids = retrieved_ids[:max_context]

            
            gi = []
            seen = set()
            for x in baseline_ids + ground_truth:
                sx = str(x)
                if sx not in seen:
                    gi.append(sx); seen.add(sx)
                if len(gi) >= max_context:
                    break
            gold_injected_ids = gi

            
            oracle_ids = ground_truth[:max_context]

            ans_vec, part_mat = None, None
            if content_selector == "grit":
                if per_sentence:
                    parts = split_sentences(answer_text)
                    part_mat = grit_encode_texts(retriever.model, parts)
                else:
                    ans_vec = grit_encode_texts(retriever.model, [answer_text])[0]

            mode_pools = {
                "baseline": baseline_ids,
                "gold_injected": gold_injected_ids,
                "oracle": oracle_ids,
            }

            for mode_name in modes:
                context_ids = mode_pools[mode_name]

                if content_selector == "grit":
                    picked_ids = select_with_grit_preencoded(
                        retriever=retriever,
                        id2idx=id2idx,
                        context_ids=context_ids,
                        ans_vec=ans_vec,
                        part_mat=part_mat,
                        top_m=top_m_citations,
                    )
                    if max_final_citations is not None:
                        picked_ids = picked_ids[:max_final_citations]
                else:
                    picked_ids = list(map(str, context_ids))  # cite all in the pool

                citations = " ".join(f"[{cid}]" for cid in picked_ids)
                attributed_answer = f"{answer_text} {citations}".strip()

                precision, recall, f1 = compute_citation_metrics(picked_ids, ground_truth)

                records.append(
                    {
                        "Query": original_query,
                        "Strategy": strategy,
                        "Mode": mode_name,
                        "Generated Answer": attributed_answer,
                        "Context IDs": context_ids,   
                        "Cited IDs": picked_ids,      
                        "Ground Truth": ground_truth,
                        "Citation Precision": precision,
                        "Citation Recall": recall,
                        "Citation F1": f1,
                    }
                )

                if pbar:
                    pbar.set_postfix({"strategy": strategy, "mode": mode_name}, refresh=False)
                    pbar.update(1)

    if pbar:
        pbar.close()

    results_df = pd.DataFrame(records)

    
    summary_rows = []
    grouped = results_df.groupby(["Strategy", "Mode"])
    for (strategy_name, mode_name), group_df in grouped:
        for metric_name in ["Citation Precision", "Citation Recall", "Citation F1"]:
            values = group_df[metric_name].tolist()
            mean_value = float(np.mean(values)) if values else 0.0
            lower_ci, upper_ci = bootstrap_confidence_interval(
                values, num_samples=bootstrap_samples, confidence_level=0.95
            )
            summary_rows.append(
                {
                    "Strategy": strategy_name,
                    "Mode": mode_name,
                    "Metric": metric_name,
                    "Mean": mean_value,
                    "CI Lower": lower_ci,
                    "CI Upper": upper_ci,
                }
            )
    summary_df = pd.DataFrame(summary_rows)
    return results_df, summary_df


In [9]:
import os
import pickle
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
from groq import Groq
from sentence_transformers import SentenceTransformer
from gritlm import GritLM

# Configuration
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
GROQ_API_KEY = ""
os.makedirs(WORKSPACE_PATH, exist_ok=True)

In [10]:
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
GRIT_INDEX_PATH = os.path.join(WORKSPACE_PATH, "grit_index.pkl")

In [15]:
from getpass import getpass
from huggingface_hub import login, whoami

token = getpass("Paste your HF token: ")  # input is hidden
login(token=token)
print(whoami())


Paste your HF token:  ········


{'type': 'user', 'id': '679ea68ce0020282a57dfeef', 'name': 'uskh580e', 'fullname': 'Usman Shafi Khan', 'isPro': False, 'avatarUrl': '/avatars/db21fbd69a85ec3f3e3eae7f9ab1e344.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'hahaha', 'role': 'fineGrained', 'createdAt': '2025-08-12T19:54:06.186Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '679ea68ce0020282a57dfeef', 'type': 'user', 'name': 'uskh580e'}, 'permissions': []}]}}}}


In [17]:
import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from groq import Groq
import os
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from groq import Groq
import time
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"


os.environ["HF_HOME"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["HF_DATASETS_CACHE"] = os.path.join(WORKSPACE_PATH, "datasets")
os.environ["NLTK_DATA"] = os.path.join(WORKSPACE_PATH, "nltk_data")


os.makedirs(os.environ["HF_HOME"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)
os.makedirs(os.environ["HF_DATASETS_CACHE"], exist_ok=True)
os.makedirs(os.environ["NLTK_DATA"], exist_ok=True)
BM25_INDEX_PATH = os.path.join(WORKSPACE_PATH, "bm25_index.pkl")
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
NLTK_PATH = os.path.join(WORKSPACE_PATH, "nltk_data")


os.makedirs(NLTK_PATH, exist_ok=True)


nltk.data.path.append(NLTK_PATH)


nltk.download("punkt", download_dir=NLTK_PATH, force=True)
nltk.download("stopwords", download_dir=NLTK_PATH, force=True)
nltk.download("wordnet", download_dir=NLTK_PATH, force=True)
print(f"Workspace set up at: {WORKSPACE_PATH}")
DATASET_CACHE_DIR = os.path.join(WORKSPACE_PATH, "huggingface_datasets")


os.makedirs(DATASET_CACHE_DIR, exist_ok=True)


query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_s2orc_data = load_dataset("princeton-nlp/LitSearch", "corpus_s2orc", split="full", cache_dir=DATASET_CACHE_DIR)

print("Datasets loaded successfully and stored in:", DATASET_CACHE_DIR)

query_df = query_data.to_pandas()
corpus_clean_df = corpus_clean_data.to_pandas()
corpus_s2orc_df = corpus_s2orc_data.to_pandas()
corpus_df = corpus_clean_df

[nltk_data] Downloading package punkt to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...


Workspace set up at: /data/horse/ws/uskh580e-myws
Datasets loaded successfully and stored in: /data/horse/ws/uskh580e-myws/huggingface_datasets


In [18]:
import numpy as np
import pandas as pd


query_df = query_df.copy()
query_df["corpusids"] = query_df["corpusids"].apply(
    lambda a: a.tolist() if isinstance(a, np.ndarray) else (a if isinstance(a, list) else [])
)
query_df["corpusids"] = query_df["corpusids"].apply(lambda xs: [int(x) for x in xs])


query_df["gold_len"] = query_df["corpusids"].apply(len)
query_df["has_gold"] = query_df["gold_len"] > 0
print(query_df["has_gold"].value_counts())
query_df[["query","corpusids"]].head(3)


has_gold
True    597
Name: count, dtype: int64


,query,corpusids
0,Are there any research papers on methods to co...,[202719327]
1,Are there any resources available for translat...,[227231792]
2,Are there any studies that explore post-hoc te...,"[226254579, 204976362]"


In [19]:
class GRITRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = GritLM("GritLM/GritLM-7B", device_map="auto", torch_dtype=torch.bfloat16, mode="embedding")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GRITLM embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing GRIT embeddings...")
        texts = [f"<|embed|>\n{text}" for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=128, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GRIT_INDEX_PATH):
            try:
                with open(GRIT_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GRIT embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GRIT_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        formatted_query = f"<|user|>\nRepresent this query for retrieving relevant documents:\n<|embed|>\n{query}"
        query_embedding = self.model.encode([formatted_query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()


grit_retriever = GRITRetriever(corpus_df)

if grit_retriever.embeddings is None:
    print("⚠️ GRITLM embeddings not found, computing...")
    grit_retriever._prepare_embeddings()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn
✅ Loaded cached GRIT embeddings


In [36]:
shuffled = query_df.sample(frac=1, random_state=41).reset_index(drop=True)
#df1, df2, df3, df4, df5 = np.array_split(shuffled, 5)

df5 = shuffled.copy()
df5 = df5[df5["corpusids"].apply(len) > 0].reset_index(drop=True)
print("Usable queries in df5:", len(df5))



Usable queries in df5: 597


In [37]:
retriever = grit_retriever

In [38]:
strategies = [
    "zero-shot",
    "few-shot",
    "chain-of-thought (Standard)",
    "self-verification",
    "chain-of-verification",
    "chain-of-citation",
    "chain-of-quote"  # include more if you like
]


In [39]:
from tqdm.auto import tqdm

In [40]:
cid_series = grit_retriever.corpus_df["corpusid"].astype(str).reset_index(drop=True)
ID2IDX = {cid: i for i, cid in enumerate(cid_series)}


In [41]:
results_df, summary_df = run_post_generation_attribution(
    query_df=df5,
    corpus_df=corpus_df,
    retriever=grit_retriever,
    strategies=strategies,
    use_llm=True,
    llm_client=client,            
    sample_fraction=1,
    k_retrieval=20,
    max_context=20,               
    retrieval_query_mode="q_plus_answer",
    modes=("baseline",),          
    content_selector="grit",
    top_m_citations=3,
    per_sentence=False,           
    max_final_citations=3,
    bootstrap_samples=2000,
    show_progress=True,
)



print("Per-query results:", len(results_df))
print("Summary rows:", len(summary_df))
results_df.head(), summary_df.head()


Post-gen attribution (Q+Answer retrieval):   0%|          | 0/4179 [00:00<?, ?it/s]

Per-query results: 4179
Summary rows: 21


(                                               Query  \
 0  Is there a specialized question answering data...   
 1  Is there a specialized question answering data...   
 2  Is there a specialized question answering data...   
 3  Is there a specialized question answering data...   
 4  Is there a specialized question answering data...   
 
                       Strategy      Mode  \
 0                    zero-shot  baseline   
 1                     few-shot  baseline   
 2  chain-of-thought (Standard)  baseline   
 3            self-verification  baseline   
 4        chain-of-verification  baseline   
 
                                     Generated Answer  \
 0  You're looking for a dataset that focuses on i...   
 1  Here's an example of how to answer the questio...   
 2  Let's break down the question step by step:\n\...   
 3  Answer:\n\nYes, there is a specialized questio...   
 4  **Draft**\n\nYes, there is a specialized quest...   
 
                                        

In [42]:
import pandas as pd

def strategy_means(summary_df, mode=None):
    df = summary_df if mode is None else summary_df[summary_df["Mode"] == mode]
    pv = (df.pivot_table(index="Strategy", columns="Metric", values="Mean")
            .reindex(columns=["Citation Precision","Citation Recall","Citation F1"])
            .sort_values("Citation F1", ascending=False)
            .round(3))
    return pv

def strategy_means_with_ci(summary_df, mode=None):
    df = summary_df if mode is None else summary_df[summary_df["Mode"] == mode]
    df = df.copy()
    df["Mean±CI"] = df.apply(lambda r: f"{r['Mean']:.3f} [{r['CI Lower']:.3f}, {r['CI Upper']:.3f}]", axis=1)
    pv = (df.pivot(index="Strategy", columns="Metric", values="Mean±CI")
            .reindex(columns=["Citation Precision","Citation Recall","Citation F1"]))
    
    order = (df[df["Metric"]=="Citation F1"]
             .sort_values("Mean", ascending=False)["Strategy"].tolist())
    return pv.loc[order]


display(strategy_means(summary_df, mode="baseline"))
display(strategy_means_with_ci(summary_df, mode="baseline"))


display(strategy_means(summary_df, mode=None))
display(strategy_means_with_ci(summary_df, mode=None))



Metric,Citation Precision,Citation Recall,Citation F1
Strategy,,,
zero-shot,0.190,0.548,0.281
chain-of-thought (Standard),0.190,0.548,0.280
chain-of-verification,0.187,0.542,0.276
few-shot,0.187,0.540,0.276
self-verification,0.183,0.529,0.270
chain-of-citation,0.179,0.518,0.265
chain-of-quote,0.160,0.461,0.236


Metric,Citation Precision,Citation Recall,Citation F1
Strategy,,,
zero-shot,"0.190 [0.176, 0.204]","0.548 [0.510, 0.589]","0.281 [0.261, 0.300]"
chain-of-thought (Standard),"0.190 [0.176, 0.203]","0.548 [0.509, 0.589]","0.280 [0.259, 0.300]"
chain-of-verification,"0.187 [0.173, 0.201]","0.542 [0.504, 0.583]","0.276 [0.257, 0.296]"
few-shot,"0.187 [0.173, 0.200]","0.540 [0.503, 0.580]","0.276 [0.256, 0.297]"
self-verification,"0.183 [0.170, 0.197]","0.529 [0.491, 0.568]","0.270 [0.251, 0.290]"
chain-of-citation,"0.179 [0.165, 0.193]","0.518 [0.480, 0.558]","0.265 [0.245, 0.285]"
chain-of-quote,"0.160 [0.145, 0.174]","0.461 [0.423, 0.499]","0.236 [0.216, 0.255]"


Metric,Citation Precision,Citation Recall,Citation F1
Strategy,,,
zero-shot,0.190,0.548,0.281
chain-of-thought (Standard),0.190,0.548,0.280
chain-of-verification,0.187,0.542,0.276
few-shot,0.187,0.540,0.276
self-verification,0.183,0.529,0.270
chain-of-citation,0.179,0.518,0.265
chain-of-quote,0.160,0.461,0.236


Metric,Citation Precision,Citation Recall,Citation F1
Strategy,,,
zero-shot,"0.190 [0.176, 0.204]","0.548 [0.510, 0.589]","0.281 [0.261, 0.300]"
chain-of-thought (Standard),"0.190 [0.176, 0.203]","0.548 [0.509, 0.589]","0.280 [0.259, 0.300]"
chain-of-verification,"0.187 [0.173, 0.201]","0.542 [0.504, 0.583]","0.276 [0.257, 0.296]"
few-shot,"0.187 [0.173, 0.200]","0.540 [0.503, 0.580]","0.276 [0.256, 0.297]"
self-verification,"0.183 [0.170, 0.197]","0.529 [0.491, 0.568]","0.270 [0.251, 0.290]"
chain-of-citation,"0.179 [0.165, 0.193]","0.518 [0.480, 0.558]","0.265 [0.245, 0.285]"
chain-of-quote,"0.160 [0.145, 0.174]","0.461 [0.423, 0.499]","0.236 [0.216, 0.255]"


In [14]:
def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts):
    """ Generates a concise answer using RAG, citing only the top 1–2 relevant sources. """
    context = "\n".join([
        f"Corpus ID: {doc_id}\nTitle: {title}\nFull Text: {full_text}"
        for doc_id, title, full_text in zip(retrieved_corpus_ids, retrieved_titles, retrieved_texts)
    ])

    prompt = f"""
Retrieved Documents:
{context}

Question: {modified_query}
Answer:"""

    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": ""},
            {"role": "user", "content": prompt}
        ]
    )


    return response.choices[0].message.content

In [13]:
def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """

    if strategy == "zero-shot":
        return query

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions while citing sources from retrieved documents.
        ...
        Now, using the retrieved documents below, answer the question while citing sources explicitly:
        {query}"""

    elif strategy == "chain-of-thought (Standard)":
        return f"""Let's break down the question step by step:
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Neutral)":
        return f"""You are an AI assistant tasked with answering a research question...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Praise)":
        return f"""You are an AI assistant known for your high-quality answers...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Punishment)":
        return f"""You are an AI assistant whose output will be evaluated...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Academic Aspiration)":
        return f"""You are a rising academic in the field of AI...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "self-verification":
        return f"""
        Step 1: Generate an initial answer based on retrieved documents.
        Step 2: Verify each claim and ensure that citations match the retrieved evidence.
        Step 3: If a claim is unsupported, explicitly state that it cannot be verified.

        Now, answer the following question:
        Question: {query}
        Answer:
        """

    elif strategy == "chain-of-verification":
        return f"""
        Let's break this down into a verification process:
        ...
        Now, answer the following question:
        Question: {query}
        Answer:
        """

    elif strategy == "chain-of-citation":
        return f"""Answer the question step by step, citing the relevant corpus ID after each reasoning step.
Use this format: [corpus: document_id]

Now, begin reasoning:
Question: {query}
Answer:"""

    elif strategy == "chain-of-quote":
        return f"""Answer the question step by step, including a short direct quote from the supporting document for each reasoning step.
Use this format: “quoted sentence” [corpus: document_id]

Now, begin reasoning:
Question: {query}
Answer:"""
